In [1]:
import unirest
import tweepy
import urllib2 as url
import pandas as pd
from IPython.display import clear_output
from time import sleep
import botometer

In [2]:
# create connection with Twitter API

CONSUMER_KEY = 'J3uhwWHdmSt3uD69ry8r2kc3B'
CONSUMER_SECRET = 'LeuM7dVfFbXH0bc5fFkZIxlNiUXyjIx4Kcjh58HeWs2TrLijDx'
ACCESS_TOKEN = '327497511-5dNjiGaTQHASljoxNr1qlMOlHrrSB21HBFBDjx2E'
ACCESS_TOKEN_SECRET = 'tHLqwlhHYJWptW3femyq0rbMW6ZItu5yLQ3DFGJqJ8Xlg'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [3]:
twitter_app_auth = {
    'consumer_key': 'J3uhwWHdmSt3uD69ry8r2kc3B',
    'consumer_secret': 'LeuM7dVfFbXH0bc5fFkZIxlNiUXyjIx4Kcjh58HeWs2TrLijDx',
    'access_token': '327497511-5dNjiGaTQHASljoxNr1qlMOlHrrSB21HBFBDjx2E',
    'access_token_secret': 'tHLqwlhHYJWptW3femyq0rbMW6ZItu5yLQ3DFGJqJ8Xlg',
  }

In [4]:
bom = botometer.Botometer(wait_on_ratelimit=True,
                          mashape_key="EIeNS0WkHxmshBQsGao1wJqAa2itp1uphHBjsn5NHYLmVvCvqs",
                          **twitter_app_auth)

In [5]:
#top claims for each month in the last 2 months

query_base = 'https://api-hoaxy.p.mashape.com/top-articles?exclude_tags=%5B%22satire%22%5D&upper_day='
articles = []
day, month, year = 15, 5, 2018

for i in range(2):
    
    if month < 10:
        date = str(year) + '-0' + str(month) + '-' + str(day)
    else:
        date = str(year) + '-' + str(month) + '-' + str(day)
    
    query = query_base + date
    
    #get top articles 
    response = unirest.get(query,
      headers={
        "X-Mashape-Key": "EIeNS0WkHxmshBQsGao1wJqAa2itp1uphHBjsn5NHYLmVvCvqs",
        "Accept": "application/json"
      }
    )

    for key in response.body['articles']:
        if key['site_type']=='claim':
            try:
                articles.append(str(key['title']))
            except:
                pass
    
    #change date
    month = month - 1
    if month == 0:
        month = 12
        year = year - 1
        
        
articles = list(set(articles))

In [6]:
len(articles)

18

In [7]:
# get ids of all the articles related to most popular articles

article_ids = []
cont = 0

for article in articles:

    cont += 1
    query = url.quote(article)
    response = unirest.get("https://api-hoaxy.p.mashape.com/articles?query="+query+"&sort_by=relevant&use_lucene_syntax=false",
      headers={
        "X-Mashape-Key": "EIeNS0WkHxmshBQsGao1wJqAa2itp1uphHBjsn5NHYLmVvCvqs",
        "Accept": "application/json"
      }
    )

    clear_output()
    print str(cont) + ' / 18'
    
    for i in range(1):
        article_ids.append(response.body['articles'][i]['id'])

        
article_ids = list(set(article_ids))

18 / 18


In [8]:
# salvataggio provvisorio lista

# ids = pd.DataFrame(article_ids)
# ids.to_csv('data/article_ids.csv',index=False)

# lettura lista salvata

# ids = pd.read_csv('data/article_ids.csv')
# article_ids = ids['0'].tolist()

In [9]:
# get tweet ids of tweets about fake news

tweet_ids = []
cont = 0

for article in article_ids:
    
    response = unirest.get("https://api-hoaxy.p.mashape.com/tweets?ids=%5B"+ str(article) + "%5D",
      headers={
        "X-Mashape-Key": "EIeNS0WkHxmshBQsGao1wJqAa2itp1uphHBjsn5NHYLmVvCvqs",
        "Accept": "application/json"
      }
    )
    
    for i in range(2000):
        
        tweet_ids.append(response.body['tweets'][i]['tweet_id'])
        
        
    cont = cont + 1
    clear_output()
    print str(cont) + ' \ 18' 
    
    
tweet_ids = [int(i) for i in tweet_ids]
tweet_ids = list(set(tweet_ids))

14 \ 18


SSLError: ('The read operation timed out',)

In [10]:
for article in article_ids[15:]:
    
    response = unirest.get("https://api-hoaxy.p.mashape.com/tweets?ids=%5B"+ str(article) + "%5D",
      headers={
        "X-Mashape-Key": "EIeNS0WkHxmshBQsGao1wJqAa2itp1uphHBjsn5NHYLmVvCvqs",
        "Accept": "application/json"
      }
    )
    
    for i in range(2000):
        
        tweet_ids.append(response.body['tweets'][i]['tweet_id'])
        
        
    cont = cont + 1
    clear_output()
    print str(cont) + ' \ 18' 
    
    
tweet_ids = [int(i) for i in tweet_ids]
tweet_ids = list(set(tweet_ids))

17 \ 18


In [6]:
# salvataggio provvisorio lista tweet

# tweet_ids = pd.DataFrame(tweet_ids)
# tweet_ids.to_csv('data/tweet_ids.csv',index=False)

# lettura lista salvata

ids = pd.read_csv('data/tweet_ids.csv')
tweet_ids = ids['0'].tolist()

In [41]:
# get ids of all the authors of the fake tweets

user_ids = []
cont = 0

for tweet in tweet_ids[:2000]:
    
    try:

        user_id = api.get_status(tweet).author.id
        user_ids.append(user_id)
    
    except:
        print Exception
    
    cont += 1
    clear_output()
    print str(cont) + ' / 34000'

2000 / 34000


In [5]:
# users = pd.DataFrame(user_ids)
# users.to_csv('data/user_ids.csv',index=False)

# users = pd.read_csv('data/user_ids.csv')
# user_ids = users['0'].tolist()

In [9]:
user_ids = []
cont = 28000

In [ ]:
for i in range(28000, 34000, 500):

    for tweet in tweet_ids[i:i+500]:
        
        cont += 1
        clear_output()
        print str(cont) + ' / 34000'
        
        try:

            user_id = api.get_status(tweet).author.id
            user_ids.append(user_id)

        except:
            print Exception
            
    print 'sleeping...'
    sleep(900)

31000 / 34000
sleeping...


In [6]:
user_ids = list(set(user_ids))

In [7]:
len(user_ids)

17372

In [15]:
# filtering of bots

bots = []
n_bots = 0
cont = 0

In [5]:
# poi riparti da 392

# prop = pd.DataFrame(bots)
# prop.to_csv('propaganda_bots.csv',index=False)

prop = pd.read_csv('data/propaganda_bots.csv')
bots = prop['0'].tolist()

In [12]:
for iid in user_ids[392:]:

    try:
        usr = bom.check_account(iid)
        cont += 1

        if usr['scores']['english'] >= 0.45:
            bots.append(iid)
            n_bots += 1
    
    except:
        pass
    
    if n_bots == 3000:
        break
        
    clear_output()
    print str(cont) + ' / 17372  ---  ' + str(n_bots) + ' bots'

18550 / 17372  ---  2745 bots


In [16]:
len(bots)

2746

In [17]:
bots = list(set(bots))

In [19]:
len(bots)

2438

In [21]:
propaganda_bots = pd.DataFrame(bots)
propaganda_bots.to_csv('data/propaganda_bots.csv',index=False)

In [22]:
len(user_ids)

19203